In [307]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time


# 명시적 대기
from selenium.webdriver.support.ui import WebDriverWait # 웹드라이버
from selenium.webdriver.support import expected_conditions as EC # 특정한 상태를 기다리는 라이브러리
from selenium.common.exceptions import ElementNotInteractableException

In [308]:
# windows 용 한글 폰트 오류 해결
from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc("font", family = font_name)

In [51]:
# 1000 개 수집하고 전처리 해서 데이터 
# 파이차트

# 데이터 수집

In [5]:
url = "https://prod.danawa.com/list/?cate=19238134&15main_19_02"

driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(3)
soup = BeautifulSoup(driver.page_source,"lxml")


In [45]:
# 각각의 상품
prod_item = soup.select("div.main_prodlist li.prod_item")
# prod_item

In [8]:
# 상품명
prod_item[0].select_one(".prod_name a").string.strip()

'로얄캐닌 미니 인도어 어덜트 8.7kg'

In [9]:
# 스펙목록
prod_item[0].select_one("div.spec-box--full div.spec_list").text.strip()

'강아지/사료/연령: 어덜트/건식/~13mm(중간알갱이)/등급: 슈퍼프리미엄/[알레르기방지]/가수분해/[효능]/효능: 위장,소화,배변, 체중조절, 구강관리, 피부,털개선/[주원료]/주원료: 과일,야채/[영양정보]/조단백: 19%/조지방: 12%/조섬유: 2.8%/조회분: 7.4%/칼슘: 0.76%/인: 0.56%/수분: 10.5%'

In [10]:
# 가격
prod_item[0].select_one(".price_sect strong").text.replace(",", "").strip()

'85080'

In [11]:
# 리뷰개수
prod_item[0].select_one(".text__number").text.replace(",", "").strip()

'9871'

In [12]:
# 별점
prod_item[0].select_one(".text__score").text

'4.7'

In [58]:
prod_data = {
    "상품명": [],
    "스펙목록": [],
    "가격": [],
    "리뷰개수": [],
    "별점": [],
}
driver = webdriver.Chrome()
driver.get(url)

driver.find_element(By.CSS_SELECTOR, ".qnt_selector").click()
driver.find_element(By.CSS_SELECTOR, ".qnt_selector option:nth-child(3)").click()

# 서울버튼을 뜨길 기다린다( 3초 )
WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "a.now_on"))
)

for i in range(10):
    time.sleep(5)
    try:
        soup = BeautifulSoup(driver.page_source, "lxml")
        prod_items = soup.select("div.main_prodlist li.prod_item")
        
        for item in prod_items:
                prod_data["상품명"].append(item.select_one(".prod_name a").string.strip())
                prod_data["스펙목록"].append(item.select_one("div.spec-box--full div.spec_list").text.strip())
                prod_data["가격"].append(item.select_one(".price_sect strong").text.replace(",", "").strip())
                try : 
                     prod_data["리뷰개수"].append(item.select_one(".text__review .text__number").text.strip())
                except:
                     prod_data["리뷰개수"].append(0)
                
                try : 
                     prod_data["별점"].append(item.select_one("span.text__score").text)
                except:
                     prod_data["별점"].append(0)
    except ElementNotInteractableException as e:
        print(e) # 에러나면 넘어가~

    if i != 9:
        pass
        driver.find_element(By.CSS_SELECTOR, ".now_on + a").click()
        


In [59]:
print(
    len(prod_data["상품명"]),
len(prod_data["스펙목록"]),
len(prod_data["가격"]),
len(prod_data["리뷰개수"]),
len(prod_data["별점"])
)

900 900 900 900 900


# 데이터 전처리

## 데이터 확인

In [538]:
df = pd.DataFrame(prod_data)

In [479]:
df.to_excel("mini_pj.xlsx", index = False)

In [539]:
df.head()

,상품명,스펙목록,가격,리뷰개수,별점
0,로얄캐닌 미니 인도어 어덜트 8.7kg,강아지/사료/연령: 어덜트/건식/~13mm(중간알갱이)/등급: 슈퍼프리미엄/[알레르...,85080,"9,871",4.7
1,나우 그레인프리 스몰브리드 어덜트 9.98kg,강아지/사료/연령: 어덜트/건식/~8mm(작은알갱이)/등급: 홀리스틱/[알레르기방지...,101590,"2,625",4.8
2,나우 그레인프리 스몰브리드 시니어 9.98kg,강아지/사료/연령: 시니어/건식/~13mm(중간알갱이)/[알레르기방지]/그레인프리/...,99810,"1,747",4.8
3,로얄캐닌 독 하이포알러제닉 스몰독 3.5kg,"강아지/사료/연령: 어덜트/건식/[알레르기방지]/가수분해/[효능]/효능: 피부,털개...",73000,"1,943",4.7
4,도그라인 아지피아 20kg,강아지/사료/연령: 주니어/건식/~8mm(작은알갱이)/등급: 그로서리/[효능]/효능...,39240,"7,332",4.7


In [481]:
df.shape

(900, 5)

In [482]:
df = pd.DataFrame.from_dict(prod_data, orient='index').T

In [483]:
df

,상품명,스펙목록,가격,리뷰개수,별점
0,로얄캐닌 미니 인도어 어덜트 8.7kg,강아지/사료/연령: 어덜트/건식/~13mm(중간알갱이)/등급: 슈퍼프리미엄/[알레르...,85080,"9,871",4.7
1,나우 그레인프리 스몰브리드 어덜트 9.98kg,강아지/사료/연령: 어덜트/건식/~8mm(작은알갱이)/등급: 홀리스틱/[알레르기방지...,101590,"2,625",4.8
2,나우 그레인프리 스몰브리드 시니어 9.98kg,강아지/사료/연령: 시니어/건식/~13mm(중간알갱이)/[알레르기방지]/그레인프리/...,99810,"1,747",4.8
3,로얄캐닌 독 하이포알러제닉 스몰독 3.5kg,"강아지/사료/연령: 어덜트/건식/[알레르기방지]/가수분해/[효능]/효능: 피부,털개...",73000,"1,943",4.7
4,도그라인 아지피아 20kg,강아지/사료/연령: 주니어/건식/~8mm(작은알갱이)/등급: 그로서리/[효능]/효능...,39240,"7,332",4.7
...,...,...,...,...,...
895,듀먼 자연화식 오리&채소 오리지널 50g,"강아지/사료/연령: 전연령/화식/[알레르기방지]/그레인프리/[효능]/효능: 위장,소...",22610,0,0
896,펫가드 보양대첩 워밍 세트 캔 30g + 파우치 80g,강아지/사료/연령: 전연령/습식/[영양정보]/조단백: 7%/조지방: 3%,16300,0,0
897,듀먼 자연화식 닭가슴살&채소 오리지널 100g,"강아지/사료/연령: 전연령/화식/[알레르기방지]/그레인프리/[효능]/효능: 위장,소...",23655,29,4.4
898,펫가드 보양대첩 워밍 캔 30g + 쿨링 캔 30g 영양식 세트,강아지/사료/연령: 전연령/습식/[영양정보]/조단백: 7%/조지방: 3%,19300,0,0


In [543]:
# 조단백 ,조지방, 조섬유, 조회분, 칼슘,인, 수분 컬럼으로 나누기
protein_list = []
fat_list = []
fiber_list = []
ash_list = []
calcium_list = []
phosphorus_list = []
moisture_list = []
# 건식/습식...
type_list = []
ingredient_list = []

# 그람수
gram_list = []

for spec_data in df["스펙목록"]:
    # "/" 기준으로 스펙 
    spec_list = spec_data.split("/")

    # 타입 추출
    # type__ = None
    # type__ = spec_list[3]
    # type_list.append(type__)

    # 사용시간이나 흡입력 값이 없을수도 있으니 None 값을 미리 입력
    protein_value = None
    fat_value = None
    fiber_value = None
    ash_value = None
    calcium_value= None
    phosphorus_value = None
    moisture_value = None
    type_value = None
    ingredient_value = None
    age_value = None
    
    for spec in spec_list:
        if "조단백:" in spec:
            protein_value = spec.split(":")[1].strip()
        elif "조지방:" in spec:
            fat_value = spec.split(":")[1].strip()
        elif "조섬유:" in spec:
            fiber_value = spec.split(":")[1].strip()
        elif "조회분:" in spec:
            ash_value = spec.split(":")[1].strip()
        elif "칼슘:" in spec:
            calcium_value = spec.split(":")[1].strip()
        elif "인:" in spec:
            phosphorus_value = spec.split(":")[1].strip()
        elif "수분:" in spec:
            moisture_value = spec.split(":")[1].strip()
        elif "주원료:" in spec:
            ingredient_value = spec.split(":")[1].strip()
        elif "연령:" in spec:
            age_value = spec.split(":")[1].strip()
        
            
        # 타입    
        if "건식" in spec:
            type_value = spec
        elif "습식" in spec:
            type_value = spec
        elif "화식" in spec:
            type_value = spec
        elif "건조" in spec:
            type_value = spec


    protein_list.append(protein_value)
    fat_list.append(fat_value)
    fiber_list.append(fiber_value)
    ash_list.append(ash_value)
    calcium_list.append(calcium_value)
    phosphorus_list.append(phosphorus_value)
    moisture_list.append(moisture_value)
    type_list.append(type_value)
    ingredient_list.append(ingredient_value)
    #연령
    age_list.append(age_value)

In [544]:
df["조단백"] = protein_list
df["조지방"] = fat_list
df["조섬유"] = fiber_list
df["조회분"] = ash_list
df["칼슘"] = calcium_list
df["인"] = phosphorus_list
df["수분"] = moisture_list
df["타입"] = type_list
df["주원료"] = ingredient_list
df["연령"] = age_list

ValueError: Length of values (4500) does not match length of index (900)

In [545]:
df.head()

,상품명,스펙목록,가격,리뷰개수,별점,조단백,조지방,조섬유,조회분,칼슘,인,수분,타입,주원료
0,로얄캐닌 미니 인도어 어덜트 8.7kg,강아지/사료/연령: 어덜트/건식/~13mm(중간알갱이)/등급: 슈퍼프리미엄/[알레르...,85080,"9,871",4.7,19%,12%,2.8%,7.4%,0.76%,0.56%,10.5%,건식,"과일,야채"
1,나우 그레인프리 스몰브리드 어덜트 9.98kg,강아지/사료/연령: 어덜트/건식/~8mm(작은알갱이)/등급: 홀리스틱/[알레르기방지...,101590,"2,625",4.8,27%,13%,4.5%,7.5%,1.2%,0.7%,10%,건식,"연어, 칠면조, 오리"
2,나우 그레인프리 스몰브리드 시니어 9.98kg,강아지/사료/연령: 시니어/건식/~13mm(중간알갱이)/[알레르기방지]/그레인프리/...,99810,"1,747",4.8,24%,10%,6%,7%,0.6%,0.4%,10%,건식,"과일,야채, 칠면조, 오리, 연어"
3,로얄캐닌 독 하이포알러제닉 스몰독 3.5kg,"강아지/사료/연령: 어덜트/건식/[알레르기방지]/가수분해/[효능]/효능: 피부,털개...",73000,"1,943",4.7,"22%, 24g","14%, 16g",1.7%,9.5%,0.648%,0.488%,10.5%,건식,"오리, 과일,야채, 돼지, 닭"
4,도그라인 아지피아 20kg,강아지/사료/연령: 주니어/건식/~8mm(작은알갱이)/등급: 그로서리/[효능]/효능...,39240,"7,332",4.7,32%,20%,3%,13%,1%,0.8%,14%,건식,"과일,야채, 닭"


In [546]:
pet_df = df

In [547]:
# 브랜드
pet_df["브랜드"] = pet_df["상품명"].str.split(" ").str[0]
pet_df["브랜드"]

0      로얄캐닌
1        나우
2        나우
3      로얄캐닌
4      도그라인
       ... 
895      듀먼
896     펫가드
897      듀먼
898     펫가드
899      듀먼
Name: 브랜드, Length: 900, dtype: object

In [548]:
pet_df["상품명"]

0                    로얄캐닌 미니 인도어 어덜트 8.7kg
1                나우 그레인프리 스몰브리드 어덜트 9.98kg
2                나우 그레인프리 스몰브리드 시니어 9.98kg
3                 로얄캐닌 독 하이포알러제닉 스몰독 3.5kg
4                           도그라인 아지피아 20kg
                      ...                 
895                 듀먼 자연화식 오리&채소 오리지널 50g
896         펫가드 보양대첩 워밍 세트 캔 30g + 파우치 80g
897              듀먼 자연화식 닭가슴살&채소 오리지널 100g
898    펫가드 보양대첩 워밍 캔 30g + 쿨링 캔 30g 영양식 세트
899                듀먼 자연화식 소고기&채소 오리지널 50g
Name: 상품명, Length: 900, dtype: object

In [549]:
pet_df["상품이름"] = pet_df["상품명"].str.split(n=1).str[1]

In [550]:
pet_df[pet_df["상품이름"].str.contains("12팩")]

,상품명,스펙목록,가격,리뷰개수,별점,조단백,조지방,조섬유,조회분,칼슘,인,수분,타입,주원료,브랜드,상품이름
41,시저 버라이어티팩 100g 12팩,강아지/사료/연령: 어덜트/습식/[효능]/효능: 체중조절/[특징]/인증: AAFCO,17150,"8,622",4.7,None,None,None,None,None,None,None,습식,None,시저,버라이어티팩 100g 12팩


In [552]:
results = []
for i in pet_df["상품명"]:
    if "kg" in i.split(" ")[-1]:
        try :
            pet_df["무게"] = pet_df["상품명"].str.split(" ").str[-1].str.split("kg").str[0].str.split("g").str[0]
        except:
            pet_df["무게"] = "100"

for i in pet_df["무게"]:
    if "g" in str(i):
        pet_df["무게"] = pet_df["무게"].str.split("g").str[0].astype(float)
    elif "ml" in str(i):
        pet_df["무게"] = pet_df["무게"].str.split("ml").str[0].astype(float)

    
pet_df["무게"]        
    # elif "g" in i.split(" ")[-1]:
    #     pet_df["무게"] = pet_df["상품명"].str.split(" ")

        

ValueError: could not convert string to float: '12팩'

In [ ]:
pet_df[100:130]

In [553]:
# 컬럼 변경
pet_selected_df = pet_df[[
    "브랜드","상품이름","가격","연령",
    
    "타입","별점","리뷰개수"
    ,"주원료"
    ,"조단백"
    ,"조지방"
    ,"조섬유"
    ,"조회분"
    ,"칼슘"
    ,"인"
    ,"수분",

    "무게"
    
]]
pet_selected_df.head()

KeyError: "['연령'] not in index"

# 전처리하기

In [554]:
pet_selected_df.dtypes

브랜드      object
상품이름     object
가격       object
연령       object
타입       object
별점      float64
리뷰개수    float64
주원료      object
조단백     float64
조지방     float64
조섬유     float64
조회분     float64
칼슘      float64
인       float64
수분      float64
무게       object
dtype: object

In [555]:
# 별점 float 로 변환
pet_selected_df["별점"] = pet_selected_df["별점"].astype(float)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_8088\3528859382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_selected_df["별점"] = pet_selected_df["별점"].astype(float)


In [556]:
try: 
    pet_selected_df["리뷰개수"] = pet_selected_df["리뷰개수"].str.replace(",","").astype(float)
except :
    pet_selected_df["리뷰개수"] = 0

C:\Users\ITSC\AppData\Local\Temp\ipykernel_8088\30437460.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_selected_df["리뷰개수"] = 0


In [557]:
pet_selected_df["조단백"] = pet_selected_df["조단백"].str.split("%").str[0]
pet_selected_df["조지방"] = pet_selected_df["조지방"].str.split("%").str[0]
pet_selected_df["조섬유"] = pet_selected_df["조섬유"].str.split("%").str[0]
pet_selected_df["조회분"] = pet_selected_df["조회분"].str.split("%").str[0]
pet_selected_df["칼슘"] = pet_selected_df["칼슘"].str.split("%").str[0]
pet_selected_df["인"] = pet_selected_df["인"].str.split("%").str[0]
pet_selected_df["수분"] = pet_selected_df["수분"].str.split("%").str[0]

AttributeError: Can only use .str accessor with string values!

In [558]:
protein = []
protein1 = []
protein2 = []
protein3 = []
protein4 = []
protein5 = []
protein6 = []

for i in pet_selected_df["조단백"]:
    try :
        protein.append(float(i))
    except:
        protein.append(0)
for i in pet_selected_df["조지방"]:
    try :
        protein1.append(float(i))
    except:
        protein1.append(0)

for i in pet_selected_df["조섬유"]:
    try :
        protein2.append(float(i))
    except:
        protein2.append(0)

for i in pet_selected_df["조회분"]:
    try :
        protein3.append(float(i))
    except:
        protein3.append(0)

for i in pet_selected_df["칼슘"]:
    try :
        protein4.append(float(i))
    except:
        protein4.append(0)

for i in pet_selected_df["인"]:
    try :
        protein5.append(float(i))
    except:
        protein5.append(0)

for i in pet_selected_df["수분"]:
    try :
        protein6.append(float(i))
    except:
        protein6.append(0)

pet_selected_df["조단백"] = protein
pet_selected_df["조지방"] = protein1
pet_selected_df["조섬유"] = protein2
pet_selected_df["조회분"] = protein3
pet_selected_df["칼슘"] = protein4
pet_selected_df["인"] = protein5
pet_selected_df["수분"] = protein6

C:\Users\ITSC\AppData\Local\Temp\ipykernel_8088\3175510201.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_selected_df["조단백"] = protein
C:\Users\ITSC\AppData\Local\Temp\ipykernel_8088\3175510201.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pet_selected_df["조지방"] = protein1
C:\Users\ITSC\AppData\Local\Temp\ipykernel_8088\3175510201.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [559]:
pet_selected_df.head()

,브랜드,상품이름,가격,연령,타입,별점,리뷰개수,주원료,조단백,조지방,조섬유,조회분,칼슘,인,수분,무게
0,로얄캐닌,미니 인도어 어덜트 8.7kg,85080,어덜트,건식,4.7,0,"과일,야채",19.0,12.0,2.8,7.4,0.760,0.560,10.5,8.7kg
1,나우,그레인프리 스몰브리드 어덜트 9.98kg,101590,어덜트,건식,4.8,0,"연어, 칠면조, 오리",27.0,13.0,4.5,7.5,1.200,0.700,10.0,9.98kg
2,나우,그레인프리 스몰브리드 시니어 9.98kg,99810,시니어,건식,4.8,0,"과일,야채, 칠면조, 오리, 연어",24.0,10.0,6.0,7.0,0.600,0.400,10.0,9.98kg
3,로얄캐닌,독 하이포알러제닉 스몰독 3.5kg,73000,어덜트,건식,4.7,0,"오리, 과일,야채, 돼지, 닭",22.0,14.0,1.7,9.5,0.648,0.488,10.5,3.5kg
4,도그라인,아지피아 20kg,39240,주니어,건식,4.7,0,"과일,야채, 닭",32.0,20.0,3.0,13.0,1.000,0.800,14.0,20kg


In [560]:
pet_selected_df.dtypes

브랜드      object
상품이름     object
가격       object
연령       object
타입       object
별점      float64
리뷰개수      int64
주원료      object
조단백     float64
조지방     float64
조섬유     float64
조회분     float64
칼슘      float64
인       float64
수분      float64
무게       object
dtype: object

In [561]:
# 저장
pet_selected_df.to_excel("mini_project_data.xlsx", index = False)

In [562]:
pet_selected_df = pd.read_excel("mini_project_data.xlsx")

# 리뷰개수가 가장 많은 상품 순위

In [563]:
pet_selected_df.head()

,브랜드,상품이름,가격,연령,타입,별점,리뷰개수,주원료,조단백,조지방,조섬유,조회분,칼슘,인,수분,무게
0,로얄캐닌,미니 인도어 어덜트 8.7kg,85080,어덜트,건식,4.7,0,"과일,야채",19.0,12.0,2.8,7.4,0.760,0.560,10.5,8.7kg
1,나우,그레인프리 스몰브리드 어덜트 9.98kg,101590,어덜트,건식,4.8,0,"연어, 칠면조, 오리",27.0,13.0,4.5,7.5,1.200,0.700,10.0,9.98kg
2,나우,그레인프리 스몰브리드 시니어 9.98kg,99810,시니어,건식,4.8,0,"과일,야채, 칠면조, 오리, 연어",24.0,10.0,6.0,7.0,0.600,0.400,10.0,9.98kg
3,로얄캐닌,독 하이포알러제닉 스몰독 3.5kg,73000,어덜트,건식,4.7,0,"오리, 과일,야채, 돼지, 닭",22.0,14.0,1.7,9.5,0.648,0.488,10.5,3.5kg
4,도그라인,아지피아 20kg,39240,주니어,건식,4.7,0,"과일,야채, 닭",32.0,20.0,3.0,13.0,1.000,0.800,14.0,20kg


In [404]:
pet_selected_df.sort_values("리뷰개수", ascending = False).head(10)

,브랜드,상품이름,가격,연령,타입,별점,리뷰개수,주원료,조단백,조지방,조섬유,조회분,칼슘,인,수분
29,뉴트리나,프라임 밸런스 15kg,12100,전연령,건식,4.6,25937.0,"과일,야채, 닭",14.0,3.0,6.0,10.0,1.00,1.10,12.0
141,천하제일,진도예찬 성견 10kg,17570,어덜트,건식,4.7,17154.0,"과일,야채",21.0,7.0,0.0,0.0,0.00,0.00,0.0
85,시저,쇠고기 100g,1570,어덜트,습식,4.7,14555.0,소,6.0,2.0,0.4,3.0,0.18,0.19,88.0
91,로얄캐닌,말티즈 어덜트 1.5kg,24260,어덜트,건식,4.7,13075.0,"돼지, 닭, 칠면조, 오리",22.0,16.0,2.4,6.1,0.50,0.50,11.0
32,ANF,6 Free 플러스 소고기&연어 5.6kg,41900,전연령,건식,4.7,11690.0,"연어, 소",27.0,10.0,6.0,8.0,1.00,0.40,11.0
0,로얄캐닌,미니 인도어 어덜트 8.7kg,85080,어덜트,건식,4.7,9871.0,"과일,야채",19.0,12.0,2.8,7.4,0.76,0.56,10.5
307,하림펫푸드,더리얼 그레인프리 크런치 닭고기 시니어 1.6kg,31000,시니어,건식,4.7,9793.0,닭,24.0,14.0,0.0,0.0,0.00,0.00,0.0
41,시저,버라이어티팩 100g 12팩,17150,어덜트,습식,4.7,8622.0,None,0.0,0.0,0.0,0.0,0.00,0.00,0.0
17,로얄캐닌,미니 인도어 어덜트 3kg,39490,어덜트,건식,4.7,8521.0,"과일,야채",19.0,12.0,2.8,7.4,0.76,0.56,10.5
7,도그랑,클래식 5kg,9090,전연령,건식,4.7,8499.0,닭,25.0,10.0,0.0,0.0,0.00,0.00,0.0


In [410]:
pet_selected_df.isnull().sum()

브랜드       0
상품이름      0
가격        0
연령      137
타입      116
별점        0
리뷰개수    387
주원료     181
조단백       0
조지방       0
조섬유       0
조회분       0
칼슘        0
인         0
수분        0
dtype: int64

In [405]:
protein_mean = pet_selected_df.groupby("타입")["조단백"].mean()

In [406]:
protein_mean.sort_values()

타입
트릿·동결건조        0.000000
습식간식           0.000000
반건조(소프트건식)     0.954545
습식             6.314414
화식            11.600000
건식            19.984716
동결건조          20.555556
소프트(반건조)      22.930000
Name: 조단백, dtype: float64

In [407]:
water_mean = pet_selected_df.groupby("타입")["수분"].mean()

In [408]:
water_mean.sort_values()

타입
동결건조           0.000000
반건조(소프트건식)     0.000000
트릿·동결건조        0.000000
습식간식           0.000000
화식             0.000000
소프트(반건조)       4.926829
건식             4.948582
습식            57.218919
Name: 수분, dtype: float64

# 내 강아지에게 먹일 최고의 사료는~~~~?

### 기준 (성분관련)
- 조단백이 20퍼가 넘김(논문 뒷받침)
- 

# 시각화
- 타입별 구성성분 분석표(비교 시각화 파트)
- 브랜드 별 구성성분 분석표(비교 시각화 파트)
- 산점도 : x축 조단백 y 축 %센또 , hue brand